In [497]:
import xmltodict
import os
import json
from sklearn.feature_extraction.text import CountVectorizer
import torch
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tqdm

In [498]:
!jupyter nbextension enable --py widgetsnbextension

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [499]:
with open('/Users/mokarakaya/develop/dataset/sem_eval/sem_eval/Restaurants_Train_v2.xml') as file:
     xmlfile = file.read()

In [500]:
sentences = json.dumps(xmltodict.parse(xmlfile))

In [501]:
sentences = json.loads(sentences)

In [502]:
sentences = sentences["sentences"]["sentence"]

In [503]:
s = []
feature = []
for sent in sentences:
#     print(sent)
    text = sent["text"]
    if "aspectTerms" in sent:
        terms = sent["aspectTerms"]["aspectTerm"]
        if type(terms) == dict:
            terms = [terms["@term"]]
        elif type(terms) == list:
            terms = [t["@term"] for t in terms]
        else: 
            raise ValueError()
    else:
        terms = []
    vec = CountVectorizer(analyzer='word', ngram_range=(1, 4)).fit([text])
    ngrams = vec.get_feature_names()
    s.extend(ngrams)
    feature.extend([ngram in terms for ngram in ngrams])
    
    
    

In [504]:
model_encoder = SentenceTransformer("sentence-transformers/paraphrase-xlm-r-multilingual-v1")

In [505]:
train_index = int(len(s) * 0.7)

In [506]:
train_s = s[:train_index]
train_f = feature[:train_index]
test_s = s[train_index:]
test_f = feature[train_index:]

In [507]:
t_false = np.argwhere(np.array(train_f) == False)[:,0]
t_remove = t_false[np.random.choice(len(t_false), size=90000, replace=False)]
t_keep = np.delete(np.arange(len(train_f)), t_remove)

In [508]:
train_s_removed = np.array(train_s)[t_keep]
train_f_removed = np.array(train_f)[t_keep]

In [509]:
encoded = model_encoder.encode(train_s_removed, show_progress_bar=True)

Batches:   0%|          | 0/152 [00:00<?, ?it/s]

In [510]:
clf = LogisticRegression(random_state=0).fit(encoded, train_f_removed)

/Users/mokarakaya/opt/anaconda3/envs/develop/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [511]:
clf.predict(model_encoder.encode(["sandwich"], show_progress_bar=True))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([ True])

In [512]:
test_s_encoded = model_encoder.encode(test_s, show_progress_bar=True)
clf.score(test_s_encoded, test_f)

Batches:   0%|          | 0/1271 [00:00<?, ?it/s]

0.9288367402159742

In [513]:
p = clf.predict(test_s_encoded)
from sklearn.metrics import confusion_matrix
confusion_matrix(test_f, p)

array([[36827,  2861],
       [   32,   933]])

In [534]:
def get_features(text,c):
    vec = CountVectorizer(analyzer='word', ngram_range=(1, 4)).fit([text])
    ngrams = vec.get_feature_names()
    predictions = c.predict(model_encoder.encode(ngrams, show_progress_bar=True))
    return zip(ngrams, predictions)
list(get_features("Best service ever", clf))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[('best', False),
 ('best service', False),
 ('best service ever', False),
 ('ever', False),
 ('service', True),
 ('service ever', False)]

In [518]:
from sklearn.neural_network import MLPClassifier
clf_mlp = MLPClassifier(random_state=1, max_iter=300).fit(encoded, train_f_removed)
list(get_features("Best service ever", clf_mlp))

/Users/mokarakaya/opt/anaconda3/envs/develop/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[('best', False),
 ('best service', False),
 ('best service ever', False),
 ('ever', False),
 ('service', True),
 ('service ever', False)]

In [549]:
p_mlp = clf_mlp.predict(predictions)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_f, p_mlp)
print(cm)
cm.astype(np.float).sum(axis=1)

[[37874  1814]
 [   48   917]]


/var/folders/y5/csvmgkln6cl70_qlk62pc7c80000gn/T/ipykernel_2681/2095902560.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cm.astype(np.float).sum(axis=1)


array([39688.,   965.])

In [520]:
clf_mlp.score(test_s_encoded, test_f)

0.9541977221853246

In [569]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
#         self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 2)
#         self.layer3 = nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(self.layer1(x))
#         x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
#         x = torch.sigmoid(self.layer3(x))
        return x



In [570]:
# train_f_p = [[0,1] if t else [1,0] for t in train_f]
# test_f_p = [[0,1] if t else [1,0] for t in test_f]

In [571]:
test_f_np = np.array(list(map(int, test_f)))

In [572]:
print(len(encoded_removed))
print(len(train_f_removed))

4857
4857


In [573]:
# X_train = Variable(torch.from_numpy(encoded)).float()
X_train = Variable(torch.from_numpy(encoded_removed)).float()
# y_train = Variable(torch.from_numpy(np.array(train_f))).long()
# y_train = Variable(torch.from_numpy(np.array(train_f))).float()
# y_train = Variable(torch.from_numpy(train_f_removed)).float()
y_train = Variable(torch.from_numpy(train_f_removed)).long()
X_test = Variable(torch.from_numpy(test_s_encoded)).float()
y_test = Variable(torch.from_numpy(test_f_np)).long()

In [574]:
sum(y_test)

tensor(965)

In [575]:
model = Model(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# loss_fn = nn.BCELoss()
# loss_fn = nn.BCEWithLogitsLoss()
loss_fn = nn.CrossEntropyLoss()

EPOCHS = 100
    

In [576]:
loss_list = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

for epoch in tqdm.trange(EPOCHS):
    y_pred = model(X_train)
#     print(y_pred[:,0])
#     loss = loss_fn(y_pred[:,0], y_train)
    loss = loss_fn(y_pred, y_train)
#     print(loss)
    loss_list[epoch] = loss.item()

    # Zero gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
#         y_pred_bin = [True if x >= 0.5 else False for x in y_pred[:,0].tolist()]
#         correct = (np.array(y_pred_bin) == np.array(y_test))
        accuracy_list[epoch] = correct.mean()
print(accuracy_list)
print(loss_list)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 16.98it/s]

[0.97621334 0.97586894 0.97296631 0.95503408 0.92738545 0.90000737
 0.8726539  0.84758812 0.82121861 0.79816985 0.78311563 0.77140677
 0.75736105 0.74481589 0.73549306 0.72678524 0.71057487 0.69925958
 0.68976462 0.68496788 0.68226206 0.67837554 0.67205369 0.66725701
 0.66401005 0.65596634 0.65362948 0.6516124  0.64971834 0.64347035
 0.64177305 0.63562346 0.63353258 0.63176149 0.63082677 0.62976903
 0.62718618 0.62536591 0.62359482 0.62192214 0.62381619 0.61867511
 0.61761737 0.61638749 0.61385381 0.61321425 0.61365706 0.61215657
 0.60974592 0.61055768 0.60878658 0.60952449 0.60492462 0.60637593
 0.59995574 0.59980816 0.59983271 0.59897178 0.59870118 0.59808624
 0.59646273 0.60010332 0.59223181 0.60305512 0.58389294 0.60669571
 0.58408976 0.59744668 0.5958724  0.5859592  0.60089046 0.585787
 0.59392911 0.5938307  0.58022779 0.5974713  0.57789093 0.58881265
 0.58610678 0.57476693 0.59257621 0.57171673 0.58074433 0.57548028
 0.57474232 0.57936686 0.57139695 0.58354855 0.5694291  0.580621

In [577]:
sum([1 if x >= 0.5 else 0 for x in y_pred[:,0].tolist()])

23053

In [581]:
y_pred = model(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_f_np, torch.argmax(y_pred, dim=1).tolist())
print(cm)
cm /  cm.astype(np.float).sum(axis=1)
# confusion_matrix(test_f_np, [1 if x >= 0.5 else 0 for x in y_pred[:,0].tolist()])

[[22507 17181]
 [  546   419]]


/var/folders/y5/csvmgkln6cl70_qlk62pc7c80000gn/T/ipykernel_2681/3691915387.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cm /  cm.astype(np.float).sum(axis=1)


array([[5.67098367e-01, 1.78041451e+01],
       [1.37573070e-02, 4.34196891e-01]])

In [579]:
loss = nn.CrossEntropyLoss()
input = torch.from_numpy(np.array([[1.0,0.0], [0.0,1.0]])).float()
print(input.size())
target = torch.from_numpy(np.array([0, 1])).long()
print(target.size())
loss(input, target).tolist()

torch.Size([2, 2])
torch.Size([2])


0.31326165795326233

In [532]:
loss =  nn.BCELoss()
input = torch.from_numpy(np.array([0,1])).float()
print(input.size())
target = torch.from_numpy(np.array([1, 1])).float()
print(target.size())
loss(input, target).tolist()

torch.Size([2])
torch.Size([2])


50.0

In [540]:
def get_features_pytorch(text,c):
    vec = CountVectorizer(analyzer='word', ngram_range=(1, 4)).fit([text])
    ngrams = vec.get_feature_names()
    X = Variable(torch.from_numpy(np.array([model_encoder.encode(ngrams, show_progress_bar=True)]))).float()
    predictions = model(X)
    return zip(ngrams, predictions[0])
list(get_features_pytorch("Best service ever", model))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[('best', tensor([0.0022, 0.6325], grad_fn=<UnbindBackward>)),
 ('best service', tensor([0.0046, 0.2798], grad_fn=<UnbindBackward>)),
 ('best service ever', tensor([0.0264, 0.0523], grad_fn=<UnbindBackward>)),
 ('ever', tensor([0.1908, 0.0091], grad_fn=<UnbindBackward>)),
 ('service', tensor([0.0861, 0.0233], grad_fn=<UnbindBackward>)),
 ('service ever', tensor([0.6900, 0.0031], grad_fn=<UnbindBackward>))]